In [ ]:
import numpy as np
import pandas as pd
import torch
!pip3 install transformers
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re

# First dataset

In [ ]:
def read_file(path):
  data = [re.split(r'\t+',line) for line in open(path, mode="r", errors="ignore").readlines() if line.strip()]
  df = pd.DataFrame(data)
  headers = df.iloc[0]
  new_df  = pd.DataFrame(df.values[1:], columns=headers)
  return new_df

train_df = read_file('2018-Valence-oc-En-train.txt')
dev_df = read_file('2018-Valence-oc-En-dev.txt')
test_df = read_file('2018-Valence-oc-En-test-gold.txt')

In [ ]:
def clean_data(df):
  df['label'] = df['Intensity Class\n'].str.split(':').str[0]
  df_clean = df.iloc[:,[1,4]]
  return df_clean

train_df = clean_data(train_df)
dev_df = clean_data(dev_df)
test_df = clean_data(test_df)
train_df.head()

In [ ]:
# train_df.to_csv (r'train_df.csv', index = False, header=True)
# from google.colab import files
# files.download("train_df.csv")

# dev_df.to_csv (r'dev_df.csv', index = False, header=True)
# files.download("dev_df.csv")

# test_df.to_csv (r'test_df.csv', index = False, header=True)
# files.download("test_df.csv")

In [ ]:
#from wordsegment import load, segment # to desegment the words
import html

def replace(text):
  text = text.replace(r"\n", " ")
  #text = re.sub(r"?\n", " ",text)
  text = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', text, flags=re.MULTILINE)
  text = re.sub(r'[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', text, flags=re.MULTILINE) # to remove other url links
  text = html.unescape(text)
  text = text.lower()
  text = re.sub(r"\’", "\'", text)
  text = re.sub(r"won\'t", "will not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"ain\'t", "am not", text)
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", "", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  
  text = re.sub(r"(rt @[-a-zA-Z0-9@:%._\+~#=]+)|(@[-a-zA-Z0-9@:%._\+~#=]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text)
  return text

train_df['Tweet'] = train_df['Tweet'].apply(lambda x: replace(x))
# train_df['Tweet']=train_df['Tweet'].apply(lambda x: replace(x))
dev_df['Tweet']=dev_df['Tweet'].apply(lambda x: replace(x))
test_df['Tweet']=test_df['Tweet'].apply(lambda x: replace(x))
train_df.head()


In [ ]:
# train_df.to_csv (r'train_df_clean.csv', index = False, header=True)
# from google.colab import files
# files.download("train_df_clean.csv")

# dev_df.to_csv (r'dev_df_clean.csv', index = False, header=True)
# files.download("dev_df_clean.csv")

# test_df.to_csv (r'test_df_clean.csv', index = False, header=True)
# files.download("test_df_clean.csv")

In [ ]:
count = train_df['Tweet'].str.split().apply(len).value_counts()
max(count)

# Second Dataset

In [ ]:
def clean_data_new(df):
  df['label'] = df['Intensity Class\n'].str.split(':').str[0]
  df_clean = df.iloc[:,[0,1,4]]
  return df_clean

In [ ]:
fear = read_file('EI-oc-En-fear-train.txt')
joy = read_file('EI-oc-En-joy-train.txt')
sadness = read_file('EI-oc-En-sadness-train.txt')
anger = read_file('EI-oc-En-anger-train.txt')
fear = clean_data_new(fear)
joy = clean_data_new(joy)
sadness = clean_data_new(sadness)
anger = clean_data_new(anger)
fear.head()

In [ ]:
fear['label'] = fear['label'].astype(int)
fear['label']  *= -1
fear.head()

In [ ]:
sadness['label'] = sadness['label'].astype(int)
sadness['label']  *= -1
sadness.head()

In [ ]:
anger['label'] = anger['label'].astype(int)
anger['label']  *= -1
anger.head()

In [ ]:
frames = [fear, joy, sadness,anger]
result = pd.concat(frames)
len(result['ID'])

In [ ]:
train_df = read_file('2018-Valence-oc-En-train.txt')
len(train_df)

In [ ]:
new_df = pd.merge(train_df,result,how="inner",on='ID')

In [ ]:
a = new_df['ID']
drop_df = result[~result['ID'].isin(a)]
len(drop_df)

In [ ]:
EI_data = drop_df.iloc[:,[1,2]]
EI_data

In [ ]:
# EI_data.to_csv (r'EI_data.csv', index = False, header=True)
# from google.colab import files
# files.download("EI_data.csv")

In [ ]:
EI_data['Tweet'] = EI_data['Tweet'].apply(lambda x: replace(x))
EI_data.head()

In [ ]:
# EI_data.to_csv (r'EI_data_clean.csv', index = False, header=True)
# from google.colab import files
# files.download("EI_data_clean.csv")

In [ ]:
# #check_df should be empty
# check_df = pd.merge(
#     drop_df,
#     train_df,
#     how="inner",
#     on='ID'
# )
# check_df 